# Лабораторная работа 100: История о данных

Обоснования для выбора набора данных: мне интересно про фильмы на самом деле и мне не так важно на чем учиться строить графики

[![Kaggle](https://img.shields.io/badge/Kaggle-035a7d?style=for-the-badge&logo=kaggle&logoColor=white)](https://www.kaggle.com/datasets/rajugc/imdb-top-250-movies-dataset)


ссылка на изначальный датасет (кнопка)





![img](../../qr-codes/kaggle-link-lab-1.png)

## Используемые библиотеки

В процессе выполнения лабораторной работы были использованы библиотеки

- pandas версии 1.5.3
- seaborn версии 0.12.2
- matplotlib 3.6.2

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from numpy import __version__ as np_version
from pandas import __version__ as pd_version
from seaborn import __version__ as sns_version
from matplotlib import __version__ as mpl_version
from plotly import __version__ as px_version

print(f"Версия plotly: {px_version}")
print(f"Версия numpy: {np_version}")
print(f"Версия pandas: {pd_version}")
print(f"Версия seaborn {sns_version}")
print(f"Версия matplotlib {mpl_version}")

Версия plotly: 5.13.1
Версия numpy: 1.23.5
Версия pandas: 1.5.3
Версия seaborn 0.12.2
Версия matplotlib 3.6.2


In [30]:
%matplotlib widget

## Первичная обработка скачанного датасета

In [ ]:
imdb_top_250_df = pd.read_csv('data/IMDB Top 250 Movies.csv')

imdb_top_250_df.head()

,rank,name,year,rating,genre,certificate,run_time,tagline,budget,box_office,casts,directors,writers
0,1,The Shawshank Redemption,1994,9.3,Drama,R,2h 22m,Fear can hold you prisoner. Hope can set you f...,25000000,28884504,"Tim Robbins,Morgan Freeman,Bob Gunton,William ...",Frank Darabont,"Stephen King,Frank Darabont"
1,2,The Godfather,1972,9.2,"Crime,Drama",R,2h 55m,An offer you can't refuse.,6000000,250341816,"Marlon Brando,Al Pacino,James Caan,Diane Keato...",Francis Ford Coppola,"Mario Puzo,Francis Ford Coppola"
2,3,The Dark Knight,2008,9.0,"Action,Crime,Drama",PG-13,2h 32m,Why So Serious?,185000000,1006234167,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",Christopher Nolan,"Jonathan Nolan,Christopher Nolan,David S. Goyer"
3,4,The Godfather Part II,1974,9.0,"Crime,Drama",R,3h 22m,All the power on earth can't change destiny.,13000000,47961919,"Al Pacino,Robert De Niro,Robert Duvall,Diane K...",Francis Ford Coppola,"Francis Ford Coppola,Mario Puzo"
4,5,12 Angry Men,1957,9.0,"Crime,Drama",Approved,1h 36m,Life Is In Their Hands -- Death Is On Their Mi...,350000,955,"Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie...",Sidney Lumet,Reginald Rose


# Поля датасета

- Rank - Ранг фильма (из топ 250 по IMDB) 
- Name - Название фильма 
- Year - Год выпуска фильма 
- Rating - Рейтинг фильма (от 0 до 10)
- genre - Жанр фильма
- certificate - Рейтинг фильма (по MPAA) 
- run_time - Продолжительность фильма
- tagline - Слоган
- budget - Бюджет фильма (в долларах США)
- box_office - Сборы фильма (по всему миру)
- directors - Режиссер(-ы) фильма
- writers - Автор(-ы) сценария

Проверка датасета на пустые значения

In [ ]:
imdb_top_250_df.isnull().sum()

rank           0
name           0
year           0
rating         0
genre          0
certificate    0
run_time       0
tagline        0
budget         0
box_office     0
casts          0
directors      0
writers        0
dtype: int64

Удаление пустых значений через ```df.dropna()``` и вывод количества строк в результирующем наборе данных

In [ ]:
imdb_top_250_df = imdb_top_250_df.dropna()
imdb_top_250_df.count()

rank           250
name           250
year           250
rating         250
genre          250
certificate    250
run_time       250
tagline        250
budget         250
box_office     250
casts          250
directors      250
writers        250
dtype: int64

Казалось, что все в этом датасете хорошо,но на самом деле...

In [ ]:
imdb_top_250_df.dtypes

rank             int64
name            object
year             int64
rating         float64
genre           object
certificate     object
run_time        object
tagline         object
budget          object
box_office      object
casts           object
directors       object
writers         object
dtype: object

Переведем продолжительность фильма из предоставленного формата в сумму минут для удобства последующего анализа и построения графиков

In [ ]:
imdb_top_250_df['run_time_m'] = imdb_top_250_df['run_time'].str.extract('(\d+)h', expand=False).astype(float)*60 + imdb_top_250_df['run_time'].str.extract('(\d+)m', expand=False).astype(float)

print(f"Средняя продолжительность фильмов в датасете {imdb_top_250_df['run_time_m'].mean()} минут")

Средняя продолжительность фильмов в датасете 129.33744855967078 минут


In [31]:
sns.distplot(a=imdb_top_250_df['run_time_m'], hist=True)

KeyError: 'run_time_m'

Новый столбец ```df_no_na["run_time_m"]``` содержит продолжительность фильма (в минутах)

Часть значений в колонках 'budget' и 'box_office' распознаются неправильно. Часть значений содержит строку Not Available

In [ ]:
box_office_not_avail = pd.DataFrame()

In [ ]:
box_office_not_avail['flag'] = imdb_top_250_df['box_office']=='Not Available'
box_office_not_avail['name'] = imdb_top_250_df['name']
new_df = box_office_not_avail.groupby(by='flag').count()
new_df

In [ ]:
px.pie(new_df)

In [ ]:
na_field_count = 0

for line in df_no_na.values:
    budget_navail = line[8] == 'Not Available'
    box_office_navail = (line[9] == 'Not Available')
    if budget_navail and box_office_navail:
        na_field_count +=1
        print(f'Для фильма "{line[1]} не известны бюджет и кассовые сборы"')

print(na_field_count)


In [ ]:
import matplotlib.pyplot as plt

plt.plot([0, 1, 2, 2])
plt

In [3]:
# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=[1, 2, 3],  # <-- Put your data instead
    y=[4, 5, 6],  # <-- Put your data instead
    z=[7, 8, 9],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [2]:
!pip install nbformat

  Using cached nbformat-5.7.3-py3-none-any.whl (78 kB)
  Using cached jsonschema-4.17.3-py3-none-any.whl (90 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 1.1 MB/s eta 0:00:00a 0:00:01


In [ ]:
df_no_na.dtypes

In [ ]:
df_no_na[df_no_na['box_office']=='Not Available'].index

Удаление всех строк без известных сборов

In [ ]:
df_no_na = df_no_na.drop(index=df_no_na[df_no_na['box_office']=='Not Available'].index).count()

Удаление всех строк без известного бюджета

In [ ]:
df_no_na = df_no_na.drop(index=df_no_na[df_no_na['budget']=='Not Available'].index).count()

In [ ]:
df_no_na['box_office_usd'] = df_no_na['box_office'].astype(float)

In [ ]:
for val in df_no_na:
    print(val)

In [ ]:
df_no_na.dtypes

In [ ]:
df.plot.scatter(x='new_runtime-', y="rating")

In [ ]:
sns.heatmap(df.corr(), annot=True, fmt='.2f')

In [ ]:
sns.boxplot( x=df["rating"], y=df["certificate"])

In [ ]:
sns.distplot( a=df["rating"], hist=True, kde=False, rug=False )

In [ ]:
df['genre']

In [ ]:
# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=[1, 2, 3],  # <-- Put your data instead
    y=[4, 5, 6],  # <-- Put your data instead
    z=[7, 8, 9],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
df.plot.scatter(x='year',
                      y='rating',
                       c='DarkBlue')

In [ ]:
sns.violinplot(x=df["rating"], y=df["year"])

# Скрипичная диаграмма по рейтингу

Для чтения и последующих манипуляций над скачанным датасетом используем
- pandas версии 1.5.3
- seaborn версии 0.12.2

Принятые сокращения:

pandas = pd
seaborn = sns

In [2]:
from pandas import __version__ as pd_ver
import pandas as pd

In [3]:
import pandas as pd


imdb_top_250_df = pd.read_csv('data/IMDB Top 250 Movies.csv')

imdb_top_250_df.head()

,rank,name,year,rating,genre,certificate,run_time,tagline,budget,box_office,casts,directors,writers
0,1,The Shawshank Redemption,1994,9.3,Drama,R,2h 22m,Fear can hold you prisoner. Hope can set you f...,25000000,28884504,"Tim Robbins,Morgan Freeman,Bob Gunton,William ...",Frank Darabont,"Stephen King,Frank Darabont"
1,2,The Godfather,1972,9.2,"Crime,Drama",R,2h 55m,An offer you can't refuse.,6000000,250341816,"Marlon Brando,Al Pacino,James Caan,Diane Keato...",Francis Ford Coppola,"Mario Puzo,Francis Ford Coppola"
2,3,The Dark Knight,2008,9.0,"Action,Crime,Drama",PG-13,2h 32m,Why So Serious?,185000000,1006234167,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",Christopher Nolan,"Jonathan Nolan,Christopher Nolan,David S. Goyer"
3,4,The Godfather Part II,1974,9.0,"Crime,Drama",R,3h 22m,All the power on earth can't change destiny.,13000000,47961919,"Al Pacino,Robert De Niro,Robert Duvall,Diane K...",Francis Ford Coppola,"Francis Ford Coppola,Mario Puzo"
4,5,12 Angry Men,1957,9.0,"Crime,Drama",Approved,1h 36m,Life Is In Their Hands -- Death Is On Their Mi...,350000,955,"Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie...",Sidney Lumet,Reginald Rose


In [8]:
import seaborn as sns

sns_ver = sns.__version__

df = imdb_top_250_df

df.dtypes

sns.violinplot(x=df['rating'],y=df['rating'])

rank             int64
name            object
year             int64
rating         float64
genre           object
certificate     object
run_time        object
tagline         object
budget          object
box_office      object
casts           object
directors       object
writers         object
dtype: object

# Диаграмма эйлера-венна по жанрам

# Barplot по годам

# Облако тэгов по жанрам

# Роза ветров по (рейтингу фильма (IMDB), продолжительности фильма, году выпуска фильма, сборам фильма, бюджету фильма)